# 詳解ディープラーニング3章

In [1]:
import numpy as np

## ロジスティック回帰

In [36]:
class LogisticRegression(object):
    '''
    ロジスティック回帰
    '''
    
    #　パラメータ初期化　このパラメータを更新していく
    def __init__(self, input_dim):
#         self.input_dim = input_dim
        self.w = np.random.normal(size=(input_dim,))
        self.b = 0.
       
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x):
        return sigmoid(np.matmul(x, self.w) + self.b)
    
    def compute_gradients(self, x, t):
        # 誤差
        delta = self.forward(x) - t
        # 勾配
        dw = np.matmul(x.T , delta)
        db = np.matmul(np.ones(x.shape[0]), delta)
        return dw, db
    
    
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [38]:
if __name__ == '__main__':
    np.random.seed(123)
    
    '''
    1. データの準備
    '''
    # or
    x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
    t = np.array([0, 1, 1, 1])
    
    '''
    2.モデルの構築
    '''
    model = LogisticRegression(input_dim=2)
    '''
    3.モデルの学習
    '''
    
    def compute_loss(t, y):
        return (-t * np.log(y) - (1 - t) * np.log(1 - y)).sum()
    
    def train_step(t, x):
        y = model(x)
        dw, db = model.compute_gradients(x, t)
        # パラメータ更新
        model.w = model.w - 0.1* dw
        model.b = model.b - 0.1 * db
#         model.w = model.w - dw
#         model.b = model.b - db
        # 損失求める←目的関数
        loss = compute_loss(t, y)
        return loss
    
    
    epochs = 100
    
    for epoch in range(epochs):
        train_loss = train_step(t, x) # バッチ学習
        
        if epoch % 10 == 0 or epoch == epochs - 1:
            print('epoch: {}, loss: {: .3f}'.format(
                epoch+1, 
                train_loss
            ))
    


epoch: 1, loss:  3.122
epoch: 11, loss:  1.823
epoch: 21, loss:  1.534
epoch: 31, loss:  1.354
epoch: 41, loss:  1.213
epoch: 51, loss:  1.097
epoch: 61, loss:  1.000
epoch: 71, loss:  0.918
epoch: 81, loss:  0.848
epoch: 91, loss:  0.788
epoch: 100, loss:  0.740


In [39]:
'''
4. モデルの評価
'''
for input in x:
    print(input)
    print(model.forward(input))

[0 0]
0.3608810957831994
[0 1]
0.8996948426020324
[1 0]
0.8438700392090897
[1 1]
0.9884869045582483


## 多クラスロジスティック回帰
+ 実装は二値分類とさほど変わらない. 重みが行列になるが、outputの次元に合わせてinputを拡張する、みたいな気持ちでいいのか

## 多層パーセプトロン
+ 非線形の分類を隠れ層を増やすことで可能にする
#### 設計
+ 入力-隠れ, 隠れ-出力の二つの層に分かれる

In [58]:
class Layer(object):
    def __init__(self, input_dim, output_dim, activation, deactivation):
        self.W = np.random.normal(size=(input_dim, output_dim))
        self.b = np.zeros(output_dim)
        self.activation = activation
        self.deactivation = deactivation
        
    def __call__(self, x):
        return self.forward(x)
    
    def forward(self, x):
        self._input = x
        self._pre_activation = np.matmul(x, self.W) + self.b
        return self.activation(self._pre_activation)
    
    def backward(self, delta, W):
        # 行きと逆の処理をする!
        delta = self.deactivation(self._pre_activation)*np.matmul(delta, W.T) 
        return delta
    
    def compute_gradients(self, delta):
        dW = np.matmul(self._input.T, delta)
        db = np.matmul(np.ones(self._input.shape[0]), delta)
        return dW, db

In [61]:
class MLP(object):
    # 各層共通のパラメータを設定
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.layer_1 = Layer(input_dim, hidden_dim, sigmoid, desigmoid)
        self.layer_2 = Layer(hidden_dim, output_dim, sigmoid, desigmoid)
        self.layers = [self.layer_1, self.layer_2]
        
    def __call__(self, x):
        return self.forward(x)
        
    def forward(self, x):
        h = self.layer_1.forward(x)
        o = self.layer_2.forward(h)
        return o 
    
        
def sigmoid(x):
    return 1/(1 + np.exp(-x))
    
def desigmoid(x):
    return sigmoid(x)*(1 - sigmoid(x))
        

In [70]:
'''
1.データ準備
'''
# xor
x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
t = np.array([[0], [1], [1], [0]])

'''
2. モデル準備
'''
model = MLP(2, 2, 1)

'''
3. 学習
'''
def compute_loss(y, t):
    return (-t * np.log(y) - (1 - t) * np.log(1 - y)).sum()

def train_step(x, t):
    y = model(x)
    # 逆伝播なので逆から回す
    for i, layer in enumerate(model.layers[::-1]):
        if i == 0:
            delta = y - t
        else:
            delta = layer.backward(delta, W)
        
        dW, db = layer.compute_gradients(delta)
        layer.W = layer.W - 0.1*dW
        layer.b = layer.b - 0.1*db
        
        W = layer.W
    
    loss = compute_loss(t, y)
    
    return loss

epochs = 1000

for epoch in range(epochs):
    train_loss = train_step(x, t)
    
    if epoch % 10 == 0 or epoch == epochs - 1:
        print('epoch: {}, loss: {: .3f}'.format(
            epoch+1, 
            train_loss
        ))
    

        

/Users/kimuramiyuki/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log


epoch: 1, loss:  inf
epoch: 11, loss:  inf
epoch: 21, loss:  inf
epoch: 31, loss:  inf
epoch: 41, loss:  inf
epoch: 51, loss:  inf
epoch: 61, loss:  inf
epoch: 71, loss:  inf
epoch: 81, loss:  inf
epoch: 91, loss:  inf
epoch: 101, loss:  inf
epoch: 111, loss:  inf
epoch: 121, loss:  inf
epoch: 131, loss:  inf
epoch: 141, loss:  inf
epoch: 151, loss:  inf
epoch: 161, loss:  inf
epoch: 171, loss:  inf
epoch: 181, loss:  inf
epoch: 191, loss:  inf
epoch: 201, loss:  inf
epoch: 211, loss:  inf
epoch: 221, loss:  inf
epoch: 231, loss:  inf
epoch: 241, loss:  inf
epoch: 251, loss:  inf
epoch: 261, loss:  inf
epoch: 271, loss:  inf
epoch: 281, loss:  inf
epoch: 291, loss:  inf
epoch: 301, loss:  inf
epoch: 311, loss:  inf
epoch: 321, loss:  inf
epoch: 331, loss:  inf
epoch: 341, loss:  inf
epoch: 351, loss:  inf
epoch: 361, loss:  inf
epoch: 371, loss:  inf
epoch: 381, loss:  inf
epoch: 391, loss:  inf
epoch: 401, loss:  inf
epoch: 411, loss:  inf
epoch: 421, loss:  inf
epoch: 431, loss:  inf

epoch: 3621, loss:  inf
epoch: 3631, loss:  inf
epoch: 3641, loss:  inf
epoch: 3651, loss:  inf
epoch: 3661, loss:  inf
epoch: 3671, loss:  inf
epoch: 3681, loss:  inf
epoch: 3691, loss:  inf
epoch: 3701, loss:  inf
epoch: 3711, loss:  inf
epoch: 3721, loss:  inf
epoch: 3731, loss:  inf
epoch: 3741, loss:  inf
epoch: 3751, loss:  inf
epoch: 3761, loss:  inf
epoch: 3771, loss:  inf
epoch: 3781, loss:  inf
epoch: 3791, loss:  inf
epoch: 3801, loss:  inf
epoch: 3811, loss:  inf
epoch: 3821, loss:  inf
epoch: 3831, loss:  inf
epoch: 3841, loss:  inf
epoch: 3851, loss:  inf
epoch: 3861, loss:  inf
epoch: 3871, loss:  inf
epoch: 3881, loss:  inf
epoch: 3891, loss:  inf
epoch: 3901, loss:  inf
epoch: 3911, loss:  inf
epoch: 3921, loss:  inf
epoch: 3931, loss:  inf
epoch: 3941, loss:  inf
epoch: 3951, loss:  inf
epoch: 3961, loss:  inf
epoch: 3971, loss:  inf
epoch: 3981, loss:  inf
epoch: 3991, loss:  inf
epoch: 4001, loss:  inf
epoch: 4011, loss:  inf
epoch: 4021, loss:  inf
epoch: 4031, los

In [71]:
'''
4. モデルの評価
'''
for input in x:
    print(input)
    print(model.forward(input))

[0 0]
[0.00486133]
[0 1]
[0.99366436]
[1 0]
[0.99369749]
[1 1]
[0.00557292]
